# Decorator

1. 함수를 받아 <strong>명령을 추가</strong>한 뒤 이를 다시 함수의 형태로 반환하는 함수이다.  
2. 함수의 내부를 수정하지 않고 기능에 변화를 주고 싶을 때 사용한다.  
3. 일반적으로 함수의 전처리나 후처리에 대한 필요가 있을때 사용을 한다.  
4. 또한 데코레이터를 이용해, 반복을 줄이고 메소드나 함수의 책임을 확장한다.

### 1. 함수형 데코레이터
함수의 결과값이 홀수이면 두배를 만들어주는 함수형 decorator 예시

In [1]:
def twiceodd(func):
    def wrapper(*args, **kwargs):
        if func(*args, **kwargs) % 2 == 1:
            return func(*args, **kwargs)
        else :
            return 2 * func(*args, **kwargs)
    return wrapper

In [2]:
@twiceodd
def add(a,b):
    return a+b
add(1, 3), add(1, 4)

(8, 5)

In [3]:
@twiceodd
def multiply(a, b):
    return a*b
multiply(1, 3), multiply(2, 5)

(3, 20)

### 2. 객체형 데코레이터 
함수의 결과값이 홀수이면 두배, 짝수이면 절반으로 쪼개는 decorator 예시

In [4]:
class TwiceOddHalfEven(object):
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        if self.func(*args, **kwargs) % 2 == 1:
            return 2 * self.func(*args, **kwargs)
        else :
            return 0.5 * self.func(*args, **kwargs)

In [5]:
@TwiceOddHalfEven
def add(a,b):
    return a+b
add(1, 3), add(1, 4)

(2.0, 10)

### 3. from functiontools import wrap
어떤 함수를 감싸는 역할을 하는 wrapper 함수를 정의 할 때 update_wrapper() 를 함수 데코레이터로써 호출하는 편리한 함수입니다.   
이것은 아래 식과 동일합니다. 
```
partial(update_wrapper, wrapped=wrapped, assigned=assigned, updated=updated)
```


### 3-1 update_wrapper()란 무엇인가 ?   
wrapper 함수를 wrapped 함수처럼 보이도록 업데이트한다. 추가되어야할 인자는 튜플의 형태로서 원래의 함수에서 

1. 원래 함수의 어떤 속성을 wrapper 함수에 매칭시킬지 
2. wrapper 함수의 어떤 속성들을 원래 함수의 값으로 업데이트 할지 명시한다. 

이 선택적 인수들은 모듈 수준의 상수인 WRAPPER_ASSIGNMENTS (__name__, __module__, __doc__) 와
WRAPPER_UPDATES (wrapper 함수의 __dict__ 즉, 인스턴스 dictionary) 이다.
    
만약 원래의 함수를 introspection(적당한 우리말 의미가 없네요.. 성찰한다는 의미) 하거나, 
다른 목적으로 접근을 허용하기 위해(예를 들어, caching decorator 인 lru_cache() 를 우회한다거나) 
즉 원래함수를 조사하고, 어떤 부분을 wrapper로 감싸는지를 지정하기 위해  
이 함수는 자동으로 __wrapped__ attribute 를 추가하여, 원래의 함수를 참조할 수 있도록 합니다
    
이 함수는 다른 함수가 원래 함수의 "함수이름", "함수 설명 텍스트", "모듈" 등의 정보를
참조할 수 있도록 전달하기 위해 필요합니다. 

In [6]:
print(add)

In [28]:
from functools import wraps

class AssignError(Exception):
    pass 

class TwiceOddorEven(object):
    def __init__(self, rule):
        self.rule = rule
    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if self.rule == 'Odd':
                if func(*args, **kwargs) % 2 == 1:
                    return 2 * func(*args, **kwargs)
                else :
                    return func(*args, **kwargs)
            elif self.rule == 'Even':
                if func(*args, **kwargs) % 2 == 0:
                    return 2 * func(*args, **kwargs)
                else :
                    return func(*args, **kwargs)
            else :
                print('Rules are Odd/Even')
                raise AssertionError
        return wrapper

In [29]:
@TwiceOddorEven(rule='Odd')
def sub(a, b):
    return a - b

In [32]:
sub(7, 4)

6

In [33]:
print(sub)

<function sub at 0x110f48f70>


위의 예시에서 볼 수 있듯이 데코레이터에 의해 add 함수의 이름/ 설명 텍스트/ 모듈 등의 정보가 변한 것을 확인할 수 있다.   
왜냐.. TwiceOddHalfEven 이라는 데코레이터 의해 add를 실행할 때, 그 안의 __call__ 함수가 반환되기 때문이다.   
즉 update_wrapper() 함수는 데코레이터가 원래함수의 "이름" / "설명 텍스트" / "모듈 정보" 등을 참조하여 사용할 수 있도록 해줍니다.

### 3-2 partial()란 무엇인가?    

원래의 함수처럼 positional arguments, keyword arguments 와 함께 호출될 수 있는 "partial object" 를 반환합니다. 더 많은 인수가 제공되면, *args* 에 추가됩니다.  
만약 추가적인 keyword arguments 가 제공되면, 기존의 keyword 를 확장하거나 덮어씁니다.  
대략적으로 아래의 함수와 같습니다.



partial object 들 .. 

* partial.func   
  callable object 또는 함수입니다. partial object 를 호출한다면, .func attribute 로 새로운 positional arguments, keyword arguments 가 전달 됩니다.    
        


        
* partial.args   
  partial object 호출에 제공될 가장 왼쪽의 positional arguments 로 positional arguments 의 앞에 추가된다.    

    
* partial.keywords   
  partial object 가 호출될 때 제공될 keyword arguments 의 dict.    
  

위의 설명만으로는 알아듣기 힘들다 다음과 예제를 확인해 보자

In [9]:
def pow(base, exponent):
    return base ** exponent
pow(2,3)

8

In [1]:
# 이때 square, cube 함수를 만들고 싶다면 이런식으로 하면된다.
def square(exponent):
    return pow(2, exponent)
def cube(exponent):
    return pow(3, exponent)
square(3), cube(4)

(8, 81)

하지만 이때 power에 대한 수많은 변수를 만들고 싶다면 어떻게 하는 것이 좋을까

In [11]:
from functools import partial

square = partial(pow, exponent=2)
cube = partial(pow, exponent=3)

assert square(2) == 4
assert cube(2) == 8
prinzt('done')

done
